### add flop/mfu stuff

I never added the flops / MFU stuff. Add that now.

In [23]:
import torch
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_checkpoint_manager import load_model
from my_nanochat.my_common import compute_init, autodetect_device_type, get_base_dir

In [7]:
device_type = autodetect_device_type() 
_, _, _, _, device = compute_init(device_type)
model, tokenizer, meta_data = load_model('base', model_tag='d20', device=device, phase='eval')

Autodetected device type: mps
loading the model from /Users/ericsilberstein/.cache/my_nanochat/base_checkpoints/d20 with step 21400
Building model with config: {'sequence_len': 2048, 'vocab_size': 65536, 'n_layer': 20, 'n_head': 10, 'n_kv_head': 10, 'n_embd': 1280}


In [8]:
f"{model.estimate_flops():,d}"

'3,491,758,080'

In [9]:
model, tokenizer, meta_data = load_model('base', model_tag='d4', device=device, phase='eval')

loading the model from /Users/ericsilberstein/.cache/my_nanochat/base_checkpoints/d4 with step 500
Building model with config: {'sequence_len': 128, 'vocab_size': 65536, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}


In [10]:
f"{model.estimate_flops():,d}"

'121,110,528'

In [21]:
# !git diff ../my_nanochat/scripts/my_base_train.py

diff --git a/my_nanochat/scripts/my_base_train.py b/my_nanochat/scripts/my_base_train.py
index 38c5bc6..fd5c966 100644
--- a/my_nanochat/scripts/my_base_train.py
+++ b/my_nanochat/scripts/my_base_train.py
@@ -121,8 +121,8 @@ orig_model = model
 model = torch.compile(model, dynamic=False)
 num_params = sum(p.numel() for p in model.parameters())
 print0(f"Number of parameters: {num_params:,}")
-# TODO num_flops_per_token = model.estimate_flops()
-# print0(f"Estimated FLOPs per token: {num_flops_per_token:e}")
+num_flops_per_token = model.estimate_flops()
+print0(f"Estimated FLOPs per token: {num_flops_per_token:e}")
 
 if num_iterations > 0:
     print0(f"Using user-provided number of iterations: {num_iterations:,}")
@@ -135,7 +135,7 @@ elif target_param_data_ratio > 0:
 total_tokens = total_batch_size * num_iterations
 print0(f"Total number of training tokens: {total_tokens:,}")
 print0(f"tokens : param ratio: {total_tokens / num_params:.2f} (he has note that Chinchilla is ~20)")
-# TOD

#### try it

In [13]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

In [22]:
!python -m scripts.my_base_train \
    --depth=4 \
    --max_seq_len=128 \
    --device_batch_size=1 \
    --num_iterations=10 \
    --total_batch_size=128 \
    --core_metric_every=0 \
    --eval_tokens=256 \
    --run=challenge-37-1

overriding depth = 4
overriding max_seq_len = 128
overriding device_batch_size = 1
overriding num_iterations = 10
overriding total_batch_size = 128
overriding core_metric_every = 0
overriding eval_tokens = 256
overriding run = challenge-37-1
user_config: {'run': 'challenge-37-1', 'device_type': '', 'depth': 4, 'max_seq_len': 128, 'num_iterations': 10, 'target_param_data_ratio': 20, 'device_batch_size': 1, 'total_batch_size': 128, 'embedding_lr': 0.2, 'unembedding_lr': 0.004, 'weight_decay': 0.0, 'matrix_lr': 0.02, 'grad_clip': 1.0, 'warmup_ratio': 0.0, 'warmdown_ratio': 0.2, 'final_lr_frac': 0.0, 'eval_every': 250, 'eval_tokens': 256, 'core_metric_every': 0, 'core_metric_max_per_task': 500, 'sample_every': 2000, 'model_tag': ''}
Autodetected device type: mps
This process is ddp_rank: 0, ddp_local_rank: 0, ddp_world_size: 1
wandb: Currently logged in as: ericsilberstein (ericsilberstein-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ setting up run h

In [27]:
!grep -i flop {get_base_dir()}/report/base-model-training.md

- Number of FLOPs per token: 1.211105e+08
- Total training flops: 1.550215e+11


### mid_train

In [29]:
# !git diff ../my_nanochat/scripts/my_mid_train.py

diff --git a/my_nanochat/scripts/my_mid_train.py b/my_nanochat/scripts/my_mid_train.py
index d60a77d..48e03ca 100644
--- a/my_nanochat/scripts/my_mid_train.py
+++ b/my_nanochat/scripts/my_mid_train.py
@@ -67,7 +67,7 @@ if pretrain_batch_size is not None and device_batch_size > pretrain_batch_size:
 orig_model = model
 model = torch.compile(model, dynamic=False)
 depth = model.config.n_layer
-# TODO num_flops_per_token
+num_flops_per_token = model.estimate_flops()
 tokens_per_fwdbwd = device_batch_size * max_seq_len # for a single rank
 world_tokens_per_fwdbwd = tokens_per_fwdbwd * ddp_world_size
 assert total_batch_size % world_tokens_per_fwdbwd == 0
@@ -167,7 +167,7 @@ ema_beta = 0.9
 total_training_time = 0
 step = 0
 while True:
-    # TODO flops_so_far
+    flops_so_far = num_flops_per_token * total_batch_size * step
 
     if ddp:
         last_step_tensor = torch.tensor(last_step, dtype=torch.int32, device=device)
@@ -186,10 +186,10 @@ while True:
             min_val_bpb = val_b

In [31]:
!python -m scripts.my_mid_train \
    --model_tag=d4 \
    --max_seq_len=128 \
    --device_batch_size=1 \
    --num_iterations=10 \
    --total_batch_size=128 \
    --eval_tokens=256 \
    --run=challenge-37-2

overriding model_tag = d4
overriding max_seq_len = 128
overriding device_batch_size = 1
overriding num_iterations = 10
overriding total_batch_size = 128
overriding eval_tokens = 256
overriding run = challenge-37-2
user_config: {'run': 'challenge-37-2', 'device_type': '', 'dtype': 'bfloat16', 'num_iterations': 10, 'max_seq_len': 128, 'device_batch_size': 1, 'unembedding_lr': 0.004, 'embedding_lr': 0.2, 'matrix_lr': 0.02, 'init_lr_frac': 1.0, 'weight_decay': 0.0, 'eval_every': 150, 'eval_tokens': 256, 'total_batch_size': 128, 'dry_run': 0}
Autodetected device type: mps
wandb: Currently logged in as: ericsilberstein (ericsilberstein-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/ericsilberstein/Documents/ericsilberstein1-repos/learn-nanochat/challenge-37-prepare-to-train-d32/wandb/run-20251222_072258

### code updated in this challenge

- my_gpt.py
- my_base_train.py
- my_mid_train.py